In [1]:
import $ivy.`org.scalanlp::breeze:1.2`
import breeze.stats._
import breeze.linalg._
import breeze.numerics.{abs, pow, sqrt}
import breeze.stats.regression.leastSquares

import scala.math._
import scala.util.Random._

import $ivy.$                         

import breeze.stats._

import breeze.linalg._

import breeze.numerics.{abs, pow, sqrt}

import breeze.stats.regression.leastSquares


import scala.math._

import scala.util.Random._

# Загрузка данных

In [2]:
val dataFile = io.Source.fromFile("/home/andy/Desktop/Industrial_ML/data/Dataset.csv")

dataFile: io.BufferedSource = non-empty iterator

In [3]:
val columns = dataFile.getLines.next.split(",")

columns: Array[String] = Array(
  "likes",
  "Checkins",
  "Returns",
  "Category",
  "commBase",
  "comm24",
  "comm48",
  "comm24_1",
  "diff2448",
  "baseTime",
  "length",
  "shares",
  "hrs",
  "sun_pub",
  "mon_pub",
  "tue_pub",
  "wed_pub",
  "thu_pub",
  "fri_pub",
  "sat_pub",
  "sun_base",
  "mon_base",
  "tue_base",
  "wed_base",
  "thu_base",
  "fri_base",
  "sat_base",
  "output"
)

Заменять NaN на нули считаю неправильным решением, для линейной модели лучше заменить отсутствующие значение на среднее значение признака, но так как мои познания scala и breeze оставляют желать лучшего, то решил оставить данное решение, так как если заменять пропущенные значения на Double.NaN, то в последствии возникают проблемы с рассчетом средних характеристик признаков.

In [4]:
val dataArray = dataFile.getLines.drop(1)
                  .map(_.split(",").map(_.trim))
                  .map { line => line.map { elem => 
                    elem match {
                      case "" => 0.0
                      case x => x.toDouble
                    }
                  }
                }.toArray

dataArray: Array[Array[Double]] = Array(
  Array(
    634995.0,
    0.0,
    463.0,
    1.0,
    3.0,
    1.0,
    2.0,
    3.0,
    -1.0,
    35.0,
    67.0,
    0.0,
    24.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0
  ),
  Array(
    634995.0,
    0.0,
    463.0,
    1.0,
    2.0,
    2.0,
    0.0,
...

In [5]:
val dataMatrix = DenseMatrix(dataArray: _*)

dataMatrix: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  3.0   1.0   2.0   3.0   -1.0   35.0  ... (28 total)
634995.0  0.0  463.0  1.0  2.0   2.0   0.0   2.0   2.0    11.0  ...
634995.0  0.0  463.0  1.0  2.0   0.0   2.0   2.0   -2.0   37.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    0.0   ...
634995.0  0.0  463.0  1.0  2.0   2.0   0.0   2.0   2.0    11.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    43.0  ...
634995.0  0.0  463.0  1.0  2.0   2.0   0.0   2.0   2.0    18.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    64.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    51.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    67.0  ...
634995.0  0.0  463.0  1.0  8.0   8.0   0.0   8.0   8.0    12.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   31.0  ...
634995.0  0.0  463.0  1.0  3.0   1.0   2.0   2.0   -1.0   38.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   0.0   -1.0   66.0  ...
634

# Немного EDA

Посмотрим на некорорые статистики целевой переменной

In [6]:
import breeze.linalg.{max, min}

println(f"Минимальное значение целевой переменной: ${min(dataMatrix(::, -1))}")
println(f"Максимальное значение целевой переменной: ${max(dataMatrix(::, -1))}")
println(f"Среднее значение целевой переменной: ${mean(dataMatrix(::, -1))}")

Минимальное значение целевой переменной: 0.0
Максимальное значение целевой переменной: 1305.0
Среднее значение целевой переменной: 7.327655726888437


import breeze.linalg.{max, min}



# Разбиение датасета на train/test 

In [7]:
val trainTestSplitRatio = 0.8
val samplesInTrain = (dataMatrix.rows * trainTestSplitRatio).toInt
val index = (0 until dataMatrix.rows).toArray
val shuffledIndex = shuffle(index)

trainTestSplitRatio: Double = 0.8
samplesInTrain: Int = 32736
index: Array[Int] = Array(
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
...
shuffledIndex: collection.mutable.ArraySeq[Int] = ArraySeq(
  24986,
  21867,
  36894,
  31278,
  14052,
  30245,
  32333,
  9997,
  39869,
  9643,
  18003,
  36836,
  18317,
  19806,
  23145,
  20651,
  6480,
  27999,
  20104,
  40706,
  16222,
  21133,
  38969,
  1735,
  8502,
  33317,
  14184,
  24639,
  31746,
  30009,
  33655,
  16048,
  7849,
  23717,
  27485,
  32237,
  31737,
  15801,
...

In [8]:
val trainIndex = shuffledIndex.slice(0, samplesInTrain).toIndexedSeq
val testIndex = shuffledIndex.slice(samplesInTrain, dataMatrix.rows).toIndexedSeq

trainIndex: IndexedSeq[Int] = Vector(
  24986,
  21867,
  36894,
  31278,
  14052,
  30245,
  32333,
  9997,
  39869,
  9643,
  18003,
  36836,
  18317,
  19806,
  23145,
  20651,
  6480,
  27999,
  20104,
  40706,
  16222,
  21133,
  38969,
  1735,
  8502,
  33317,
  14184,
  24639,
  31746,
  30009,
  33655,
  16048,
  7849,
  23717,
  27485,
  32237,
  31737,
  15801,
...
testIndex: IndexedSeq[Int] = Vector(
  6792,
  33361,
  33309,
  3477,
  31152,
  34921,
  7397,
  14533,
  30877,
  59,
  23011,
  35882,
  9815,
  21470,
  4379,
  35652,
  38200,
  38173,
  14697,
  33647,
  29695,
  21140,
  25024,
  246,
  4148,
  243,
  26572,
  5688,
  22594,
  20272,
  7,
  14007,
  9071,
  36968,
  26837,
  9802,
  23459,
  2152,
...

In [9]:
val XTrain = dataMatrix(trainIndex, 0 to 26)
val XTest = dataMatrix(testIndex, 0 to 26)

val yTrain = dataMatrix(trainIndex, 27).toDenseVector
val yTest = dataMatrix(testIndex, 27).toDenseVector

XTrain: SliceMatrix[Int, Int, Double] = 1045111.0    0.0       34768.0    16.0  6.0    6.0    0.0    ... (27 total)
209071.0     0.0       444.0      55.0  3.0    3.0    0.0    ...
1079539.0    0.0       66429.0    18.0  250.0  250.0  0.0    ...
2078654.0    2715.0    84453.0    9.0   100.0  100.0  0.0    ...
25599.0      2023.0    1835.0     32.0  0.0    0.0    0.0    ...
41813.0      0.0       10840.0    2.0   143.0  4.0    20.0   ...
98168.0      0.0       1843.0     36.0  25.0   1.0    2.0    ...
4233.0       0.0       1005.0     18.0  7.0    0.0    1.0    ...
2156275.0    114.0     47233.0    9.0   49.0   3.0    23.0   ...
1872.0       0.0       0.0        16.0  0.0    0.0    0.0    ...
25879.0      0.0       1970.0     36.0  48.0   0.0    6.0    ...
350630.0     0.0       66944.0    18.0  385.0  87.0   298.0  ...
18236.0      0.0       81.0       4.0   3.0    0.0    1.0    ...
5776.0       294.0     60.0       27.0  0.0    0.0    0.0    ...
27279.0      9.0       125.0      18.0 

# Сделаем min max scaling

In [10]:
import breeze.linalg.{max, min}

val featureMax = max(XTrain(::, *)).t.toArray
val featureMin = min(XTrain(::, *)).t.toArray

import breeze.linalg.{max, min}


featureMax: Array[Double] = Array(
  4.86972297E8,
  186370.0,
  6089942.0,
  106.0,
  2341.0,
  2079.0,
  1592.0,
  2184.0,
  2079.0,
  72.0,
  21480.0,
  144860.0,
  24.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0
)
featureMin: Array[Double] = Array(
  36.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1366.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0
)

In [11]:
def createMatrixFromArray(statistics: Array[Double], nRows: Int) = { 
    var returnVal = List[Array[Double]]()
    for (i <- 1 to nRows) returnVal :+= statistics
    returnVal.toArray
}

defined function createMatrixFromArray

In [12]:
val maxValuesTrain = DenseMatrix(createMatrixFromArray(featureMax, XTrain.rows): _*)
val minValuesTrain = DenseMatrix(createMatrixFromArray(featureMin, XTrain.rows): _*)

val maxValuesTest = DenseMatrix(createMatrixFromArray(featureMax, XTest.rows): _*)
val minValuesTest = DenseMatrix(createMatrixFromArray(featureMin, XTest.rows): _*)

maxValuesTrain: DenseMatrix[Double] = 4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ... (27 total)
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.0  6089942.0  106.0  2341.0  2079.0  ...
4.86972297E8  186370.

In [13]:
val scaledXTrain = (XTrain - minValuesTrain).toDenseMatrix /:/ (maxValuesTrain - minValuesTrain).toDenseMatrix
val scaledXTest = (XTest - minValuesTest).toDenseMatrix /:/ (maxValuesTest - minValuesTest).toDenseMatrix

scaledXTrain: DenseMatrix[Double] = 0.0021460667961947836  0.0                    ... (27 total)
4.2925442934007283E-4  0.0                    ...
0.0022167648682560174  0.0                    ...
0.004268452572086031   0.01456779524601599    ...
5.249374974152789E-5   0.010854751301175082   ...
8.578928071634043E-5   0.0                    ...
2.015145581361974E-4   0.0                    ...
8.618560719211067E-6   0.0                    ...
0.0044278476880226245  6.116864302194559E-4   ...
3.770235282456879E-6   0.0                    ...
5.306873115715312E-5   0.0                    ...
7.199465515346879E-4   0.0                    ...
3.7373792015640087E-5  0.0                    ...
1.1787119020317257E-5  0.0015775071095133336  ...
5.594363823527928E-5   4.8291033964693887E-5  ...
5.4458954079932696E-6  0.027354187905778826   ...
0.0029208871098306767  0.0                    ...
0.004109394230978589   0.0                    ...
1.0375744995462894E-4  0.055577614422922146   ...
6.3

# Построение модели линейной регрессии, используя стандартные функции Breeze

In [14]:
val scaledXTrainWithIntercept = DenseMatrix.horzcat(scaledXTrain, DenseVector.ones[Double](scaledXTrain.rows).toDenseMatrix.t)

scaledXTrainWithIntercept: DenseMatrix[Double] = 0.0021460667961947836  0.0                    ... (28 total)
4.2925442934007283E-4  0.0                    ...
0.0022167648682560174  0.0                    ...
0.004268452572086031   0.01456779524601599    ...
5.249374974152789E-5   0.010854751301175082   ...
8.578928071634043E-5   0.0                    ...
2.015145581361974E-4   0.0                    ...
8.618560719211067E-6   0.0                    ...
0.0044278476880226245  6.116864302194559E-4   ...
3.770235282456879E-6   0.0                    ...
5.306873115715312E-5   0.0                    ...
7.199465515346879E-4   0.0                    ...
3.7373792015640087E-5  0.0                    ...
1.1787119020317257E-5  0.0015775071095133336  ...
5.594363823527928E-5   4.8291033964693887E-5  ...
5.4458954079932696E-6  0.027354187905778826   ...
0.0029208871098306767  0.0                    ...
0.004109394230978589   0.0                    ...
1.0375744995462894E-4  0.055577614422922

In [15]:
val resultBuiltinImpl = leastSquares(scaledXTrainWithIntercept, yTrain.toDenseVector)

Sep 14, 2021 11:58:22 PM com.github.fommil.netlib.LAPACK <clinit>
Sep 14, 2021 11:58:22 PM com.github.fommil.netlib.LAPACK <clinit>


resultBuiltinImpl: regression.LeastSquaresRegressionResult = LeastSquaresRegressionResult(
  DenseVector(-51.709057229803285, -3.920044983644351, 89.81595546296593, -4.093513393013508, 453.239856413774, 1645.6917919339587, -906.7708317820124, -456.02577057755997, -1893.491320179177, -14.115965446137949, 9.13378012538764, 216.5400590654455, 9.834885450981817, -0.6106646587426139, -0.7506854120250583, -1.199241478450213, 0.34101085872245795, 0.13840114901233672, 0.5857754294070499, -0.6316437433304537, -0.8637171197878214, -0.2721691392902364, -0.90123095993736, 0.699164341111676, -0.3000815034171319, -0.10771352656946197, -1.312086481053364, 751.5086710963964),
  2.9780366781932727E7
)

In [16]:
val nFeatures = XTrain.cols

nFeatures: Int = 27

In [17]:
val interceptBuiltinImpl = resultBuiltinImpl.coefficients(-1)
val coefsBuiltinImpl = resultBuiltinImpl.coefficients.slice(0, nFeatures).toArray

interceptBuiltinImpl: Double = 751.5086710963964
coefsBuiltinImpl: Array[Double] = Array(
  -51.709057229803285,
  -3.920044983644351,
  89.81595546296593,
  -4.093513393013508,
  453.239856413774,
  1645.6917919339587,
  -906.7708317820124,
  -456.02577057755997,
  -1893.491320179177,
  -14.115965446137949,
  9.13378012538764,
  216.5400590654455,
  9.834885450981817,
  -0.6106646587426139,
  -0.7506854120250583,
  -1.199241478450213,
  0.34101085872245795,
  0.13840114901233672,
  0.5857754294070499,
  -0.6316437433304537,
  -0.8637171197878214,
  -0.2721691392902364,
  -0.90123095993736,
  0.699164341111676,
  -0.3000815034171319,
  -0.10771352656946197,
  -1.312086481053364
)

In [18]:
var coefsMatrix = DenseMatrix(createMatrixFromArray(coefsBuiltinImpl, XTest.rows): _*)

coefsMatrix: DenseMatrix[Double] = -51.709055784718196  -3.9200448842420044  89.81595460467496  ... (27 total)
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
-51.709055784718196  -3.9200448842420044  89.81595460467496  ...
...

In [19]:
var featureCoefMul = scaledXTest *:* coefsMatrix
var yPred = sum(featureCoefMul(*, ::)) + interceptBuiltinImpl

featureCoefMul: DenseMatrix[Double] = -0.00450266057658506    -0.0                    ... (27 total)
-0.003274951832445911   -0.0                    ...
-0.031098875492108043   -0.0                    ...
-0.3187657302212665     -0.0                    ...
-0.003896770065076845   -0.0                    ...
-0.1594739716754552     -0.0                    ...
-0.0015957452868814329  -0.0                    ...
-0.005891557858486121   -0.0                    ...
-0.055092925515408744   -0.0                    ...
-0.06742299917573515    -0.0                    ...
-9.072429951500507E-4   -0.0                    ...
-0.00953996165885604    -0.008266231901631742   ...
-0.006217969956694367   -0.002292669916737557   ...
-0.024207056747115194   -0.0                    ...
-0.07139600993258607    -2.1033668960895016E-5  ...
-7.385153360567185E-4   -0.0                    ...
-0.0057625433173921     -0.0                    ...
-0.0057625433173921     -0.0                    ...
-0.03878708029323682    -0.0                    ...
-0.06721285944172276    -0.003849161419843788   ...
-0.09189243858214527    -0.0                    ...
-0.023436367414390584   -0.0                    ...
-0.02463731758704357    -0.0                    ...
-0.08479770067004898    -0.0                    ...
-0.07139600993258607    -2.1033668960895016E-5  ...
-0.08479770067004898    -0.0                    ...
-0.009431016046376646   -0.0                    ...
-0.012292802792899238   -0.0                    ...
-0.3220007565660981     -0.09641833851674277    ...
-0.0026806354649535386  -0.0058894273090506054  ...
-0.06742299917573515    -0.0                    ...
-0.002714402233733702   -0.042551112307890625   ...
-8.728391175249787E-5   -0.0                    ...
-0.0359925085298521     -0.0                    ...
-0.09339474123844008    -2.3137035856984518E-4  ...
-0.006217969956694367   -0.002292669916737557   ...
-1.571535150774901E-5   -0.0                    ...
-0.034993309491420224   -0.0                    ...
-8.503279383382029E-4   -0.0041646664542572135  ...
...
yPred: DenseVector[Double] = DenseVector(11.766087020911186, 4.612865108207416, 7.625539577691484, -3.00356223083827, -0.8899950915221098, 2.090525082559111, 5.303515703210451, 8.872748401987451, 6.042738408247487, 1.7962980114529046, 5.289327727386876, -0.6549314358429683, 12.999153034617962, 8.469429358908997, 9.057322551973698, 0.15252849434489235, -1.5399379056794942, -0.32766084045465504, -2.2909431660393693, 28.01206377559913, 3.3104190530776805, 9.776226008546473, -2.0413013658867385, 16.676197938556697, 0.5645199386009381, 7.838125463861729, 4.8139137672811785, 7.819186930610044, -0.11023413446639552, -5.496295911734023, -1.7069126196186062, 9.902932694765582, 8.01655248722443, -3.1944883559432355, -3.0863936552556197, 6.940024632340737, 3.6906227227191266, -3.598199640991538, -0.6864953176508379, -2.8043550947941185, 0.5804655905719756, 2.1941367323134955, -1.2792786616347485, -3.3824245965705586, 8.492305323984624, 11.746200467400172, 5.729791402920455, 5.0853667581583295, 2.6104903292656445, 5.664718478812233, 3.098711903553067, 9.699284113382873, 2.6623620269671164, -3.467045527360824, 1.1549377298576928, -0.6282942559892035, 17.468754542986744, 9.381484070452188, 3.8357272149692108, -0.0996894336093419, 9.419507691172726, 0.12252260011496219, 1.3816971385315355, 5.1389285802164295, 26.09014719944082, 3.376692222411748, 2.1933561231051044, 14.130075622616914, 3.5962333783555778, -2.4758333800122045, 0.3109834026926137, -3.4596928230103003, -1.0383726866092502, -3.4997555437856818, 10.317221594690977, -3.821219114790324, -7.073029215170891, 7.042835403090066, 0.13810241022110858, 6.058271779699794, -6.25918444961917, -1.64567476473826, -3.149286836541478, -1.9682640270601723, 2.4812588015497568, 1.8802990973599663, 1.4908775363650193, 7.067032120020031, 9.082065024865415, 81.82483027556088, -1.9410895309403031, 2.0952033007923774, 1.4098172143148986, 5.176250311610033, -1.899

In [20]:
def calculateRMSE(yTrue: DenseVector[Double], yPred: DenseVector[Double]) = { 
    sqrt(sum((yTrue - yPred).map(x => x*x)) / yTrue.length)
}

def calculateMAE(yTrue: DenseVector[Double], yPred: DenseVector[Double]) = { 
    sum((yTrue - yPred).map(abs)) / yTrue.length
}

defined function calculateRMSE
defined function calculateMAE

In [21]:
println(f"RMSE до обработки предсказаний: ${calculateRMSE(yTest, yPred)}")
println(f"MAE до обработки предсказаний:: ${calculateMAE(yTest, yPred)}")

RMSE до обработки предсказаний: 27.8067874973524
MAE до обработки предсказаний:: 8.034724141302805


Занулим все отрицательные предсказанные значения

In [22]:
var positiveYPred = yPred.map(x => if (x >= 0.0) x else 0.0)

positiveYPred: DenseVector[Double] = DenseVector(11.766087020911186, 4.612865108207416, 7.625539577691484, 0.0, 0.0, 2.090525082559111, 5.303515703210451, 8.872748401987451, 6.042738408247487, 1.7962980114529046, 5.289327727386876, 0.0, 12.999153034617962, 8.469429358908997, 9.057322551973698, 0.15252849434489235, 0.0, 0.0, 0.0, 28.01206377559913, 3.3104190530776805, 9.776226008546473, 0.0, 16.676197938556697, 0.5645199386009381, 7.838125463861729, 4.8139137672811785, 7.819186930610044, 0.0, 0.0, 0.0, 9.902932694765582, 8.01655248722443, 0.0, 0.0, 6.940024632340737, 3.6906227227191266, 0.0, 0.0, 0.0, 0.5804655905719756, 2.1941367323134955, 0.0, 0.0, 8.492305323984624, 11.746200467400172, 5.729791402920455, 5.0853667581583295, 2.6104903292656445, 5.664718478812233, 3.098711903553067, 9.699284113382873, 2.6623620269671164, 0.0, 1.1549377298576928, 0.0, 17.468754542986744, 9.381484070452188, 3.8357272149692108, 0.0, 9.419507691172726, 0.12252260011496219, 1.3816971385315355, 5.1389285802164295, 26.09014719944082, 3.376692222411748, 2.1933561231051044, 14.130075622616914, 3.5962333783555778, 0.0, 0.3109834026926137, 0.0, 0.0, 0.0, 10.317221594690977, 0.0, 0.0, 7.042835403090066, 0.13810241022110858, 6.058271779699794, 0.0, 0.0, 0.0, 0.0, 2.4812588015497568, 1.8802990973599663, 1.4908775363650193, 7.067032120020031, 9.082065024865415, 81.82483027556088, 0.0, 2.0952033007923774, 1.4098172143148986, 5.176250311610033, 0.0, 21.739348584444656, 0.0, 0.0, 3.0012536561273464, 0.0, 0.0, 130.66619778561778, 0.0, 2.57515524000155, 8.087652804095455, 3.7461993745258724, 0.0, 64.81977749908003, 0.0, 0.0, 0.0, 62.29040712216431, 1.9004617071940402, 0.0, 1.2801874680169476, 0.0, 5.356078724681311, 30.783411134087487, 16.86068069383657, 7.32275906659595, 16.950942917582665, 1.717280124521949, 0.0, 0.6177819039946826, 2.723141578438913, 0.0, 7.968460963681082, 7.388209395523859, 0.0, 0.0, 23.505050135064153, 3.896050263331176, 5.250131183401436, 0.0, 0.0, 1.99897714899339, 7.322110398498808, 0.0, 3.4774904102499704, 2.5093092041583986, 0.0, 11.915470208019656, 0.0, 0.0, 2.207271288545826, 10.404103219154877, 0.044697710871673735, 7.642562993632964, 0.0, 0.0, 8.27978915844642, 3.540021592055723, 12.051654741045127, 7.686507743688821, 27.92019585075002, 17.119946323966133, 8.844029091105767, 6.690492903130121, 39.68500715049822, 0.0, 0.0, 4.974440291648534, 0.0, 49.378395597817985, 4.152770114454938, 0.0, 7.370662794726741, 5.837816216762008, 16.726877402669288, 0.0, 9.02986628706094, 7.026423724239066, 13.057478943084675, 10.768662905064161, 7.502739163640399, 0.0, 4.078280976200176, 9.076557654384828, 0.0, 42.55658083029812, 0.0, 16.96078589376191, 7.496717019734319, 6.482340482612358, 2.5730290295196028, 7.781646814458668, 11.734858748124907, 4.905909337638491, 7.249801332470497, 0.027522402464114748, 7.813365087297029, 0.8227069684049866, 0.0, 3.9875840165444743, 3.1776489641169974, 0.0, 7.966449250211781, 12.200617528054863, 0.0, 0.0, 119.94767367400311, 0.0, 11.27140608042589, 1.3388366627116284, 21.86556118889939, 0.0, 19.750046721683702, 2.415542378299051, 3.6766775677338046, 15.608034324015193, 25.725658297124255...

In [23]:
println(f"RMSE после обработки предсказаний: ${calculateRMSE(yTest, positiveYPred)}")
println(f"MAE после обработки предсказаний: ${calculateMAE(yTest, positiveYPred)}")

RMSE после обработки предсказаний: 27.732885903334413
MAE после обработки предсказаний: 7.169059841004454


# Реализация аналитического решения задачи линейной регрессии

In [24]:
val result = inv(scaledXTrainWithIntercept.t * scaledXTrainWithIntercept) * scaledXTrainWithIntercept.t * yTrain

Sep 14, 2021 11:58:23 PM com.github.fommil.netlib.BLAS <clinit>
Sep 14, 2021 11:58:23 PM com.github.fommil.netlib.BLAS <clinit>


result: DenseVector[Double] = DenseVector(-51.709055784718196, -3.9200448842420044, 89.81595460467496, -4.09351365801777, 453.23985464826063, 1646.5342191476957, -907.4159244879411, -456.02576878049354, -1894.8872714933823, -14.115965726831565, 9.13378208228004, 216.54005818599074, 9.834885486309043, -0.610663740163648, -0.7506848570981961, -1.199240603544065, 0.34101172413804215, 0.13840209419150964, 0.5857764291532687, -0.6316427727603658, -0.8637174368818908, -0.27216913927369524, -0.9012308392296958, 0.6991642229285145, -0.300081407941451, -0.10771351352682593, -1.3120865051793151, 752.0621868702292)

In [25]:
val interceptManualImpl = result(-1)
val coefsManualImpl = result.slice(0, nFeatures).toArray

interceptManualImpl: Double = 752.0621868702292
coefsManualImpl: Array[Double] = Array(
  -51.709055784718196,
  -3.9200448842420044,
  89.81595460467496,
  -4.09351365801777,
  453.23985464826063,
  1646.5342191476957,
  -907.4159244879411,
  -456.02576878049354,
  -1894.8872714933823,
  -14.115965726831565,
  9.13378208228004,
  216.54005818599074,
  9.834885486309043,
  -0.610663740163648,
  -0.7506848570981961,
  -1.199240603544065,
  0.34101172413804215,
  0.13840209419150964,
  0.5857764291532687,
  -0.6316427727603658,
  -0.8637174368818908,
  -0.27216913927369524,
  -0.9012308392296958,
  0.6991642229285145,
  -0.300081407941451,
  -0.10771351352682593,
  -1.3120865051793151
)

In [26]:
coefsMatrix = DenseMatrix(createMatrixFromArray(coefsManualImpl, XTest.rows): _*)

In [27]:
featureCoefMul = scaledXTest *:* coefsMatrix
yPred = sum(featureCoefMul(*, ::)) + interceptManualImpl

In [28]:
println(f"RMSE до обработки предсказаний: ${calculateRMSE(yTest, yPred)}")
println(f"MAE до обработки предсказаний:: ${calculateMAE(yTest, yPred)}")

RMSE до обработки предсказаний: 27.806787489166528
MAE до обработки предсказаний:: 8.034723890233128


In [29]:
positiveYPred = yPred.map(x => if (x >= 0.0) x else 0.0)

In [30]:
println(f"RMSE после обработки предсказаний: ${calculateRMSE(yTest, positiveYPred)}")
println(f"MAE после обработки предсказаний: ${calculateMAE(yTest, positiveYPred)}")

RMSE после обработки предсказаний: 27.73288583890136
MAE после обработки предсказаний: 7.16905908404035
